In [ ]:
from jira import JIRA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

jira_url = 'https://kainos-evolve.atlassian.net'
jira = JIRA(jira_url)

In [ ]:
startDate = '2018-01-01'
endDate = '2018-02-01'

In [ ]:
jql = 'project = vxt and status = Completed AND status changed to Completed during ({}, {}) \
 AND type in (bug, "External Bug", story)'.format(startDate, endDate)

#jql = 'project = vxt and type = bug'

jql

issuesRaw = jira.search_issues(jql, maxResults = False)

issues = pd.DataFrame()
issues['key'] = ''
issues['category'] = ''
issues['type'] = ''
issues['team'] = ''
issues['SP'] = 0
issues['bugEffort'] = ''
issues['status'] = ''
issues['summary'] = ''
issues['month'] = ''

for issue in issuesRaw:
    try:
        bugEffort = issue.fields.customfield_14703
    except AttributeError:
        bugEffort = np.nan
    
    try:
        team = issue.fields.customfield_14200.value
    except AttributeError:
        team = np.nan
        
    try:
        category = str(issue.fields.customfield_14709.value)
    except AttributeError:
        category = 'Not set'
    
    issues = issues.append(
        {
         'key': issue.key,
         'type': issue.fields.issuetype.name,
         'status': issue.fields.status.name,
         'category': issue.fields.customfield_14709,
         'bugEffort': bugEffort,
         'SP' : issue.fields.customfield_10005,
         'summary': issue.fields.summary,
         'team': team
        }, ignore_index=True)
        

issues['month'] = 'Jan'
issues = issues.loc[(issues['team'].isin(["Gdansk Team 1", "Gdansk Team 2", "Belfast Team"]))]
issues

In [ ]:
issues.loc[(issues['type'] == 'Bug') & (issues['SP'].isnull()), ["SP"]] = issues.bugEffort
issues['SP']=issues['SP'].fillna(0)
#issues['category']=issues['category'].fillna('Not set')
issues['category'] = issues.category.astype(str)
issues


In [ ]:
#print(issues['type'].dtype)
issuesByCategory = issues.groupby(['category']).agg({'SP':'sum'})

issuesByCategory
#issuesByCategory = issuesByCategory.reset_index()

